<a href="https://colab.research.google.com/github/emmastoklee/levelingtheplayingfield/blob/main/Testing_BERTopic_for_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 4.0 MB/s eta 0:00:00


In [35]:
pip install openai


In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import OpenAI
import openai


In [52]:
import torch
print(torch.cuda.is_available())  # Should return True if a GPU is available


True


In [41]:
print(openai.__version__)

1.46.1


In [3]:
# mount on drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/My Drive/SDS/Thesis/Data/guardian_test_data_2020-2024.csv')
df.head()

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,...,commentCloseDate,commentable,displayHint,liveBloggingNow,sensitive,starRating,scheduledPublicationDate,showTableOfContents,clean_text,womens_football
0,football/2020/dec/31/kolasinac-on-verge-of-sch...,article,football,Football,2020-12-31T22:30:04Z,Kolasinac joins Schalke on loan as Arteta targ...,https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['mikel', 'arteta', 'has', 'revealed', 'arsena...",False
1,football/2020/dec/31/talented-charismatic-vola...,article,football,Football,2020-12-31T21:06:46Z,"Tommy Docherty: colourful, complex coach with ...",https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,2021-01-03T21:00:00Z,True,NaN,NaN,NaN,NaN,NaN,NaN,"['tommy', 'docherty', 'spent', 'a', 'lifetime'...",False
2,football/2020/dec/31/tommy-docherty-obituary,article,football,Football,2020-12-31T20:42:14Z,Tommy Docherty obituary,https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,2021-01-03T20:42:14Z,True,NaN,NaN,NaN,NaN,NaN,NaN,"['tommy', 'docherty', 'who', 'has', 'died', 'a...",False
3,football/ng-interactive/2020/dec/31/mens-trans...,interactive,football,Football,2020-12-31T20:00:49Z,Men's transfer window January 2021 – all deals...,https://www.theguardian.com/football/ng-intera...,https://content.guardianapis.com/football/ng-i...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['default'],False
4,football/2020/dec/31/kieran-trippier-suspensio...,article,football,Football,2020-12-31T19:33:39Z,Kieran Trippier's suspension by FA 'completely...,https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['diego', 'simeone', 'has', 'described', 'the'...",False


In [5]:
# split df into df_men and df_women based on womens_football column
df_men = df[df['womens_football'] == 0]
df_women = df[df['womens_football'] == 1]

In [6]:
len(df_men)

12759

In [7]:
len(df_women)

3500

In [8]:
# display list of column headings
df.columns.tolist()


['id',
 'type',
 'sectionId',
 'sectionName',
 'webPublicationDate',
 'webTitle',
 'webUrl',
 'apiUrl',
 'isHosted',
 'pillarId',
 'pillarName',
 'headline',
 'standfirst',
 'trailText',
 'byline',
 'main',
 'body',
 'newspaperPageNumber',
 'wordcount',
 'firstPublicationDate',
 'isInappropriateForSponsorship',
 'isPremoderated',
 'lastModified',
 'newspaperEditionDate',
 'productionOffice',
 'publication',
 'shortUrl',
 'shouldHideAdverts',
 'showInRelatedContent',
 'thumbnail',
 'legallySensitive',
 'lang',
 'isLive',
 'bodyText',
 'charCount',
 'shouldHideReaderRevenue',
 'showAffiliateLinks',
 'bylineHtml',
 'commentCloseDate',
 'commentable',
 'displayHint',
 'liveBloggingNow',
 'sensitive',
 'starRating',
 'scheduledPublicationDate',
 'showTableOfContents',
 'clean_text',
 'womens_football']

In [ ]:
# show example of body
df['body'][0]

'<p>Mikel Arteta has revealed Arsenal’s priority in January is to trim the numbers in their squad. Sead Kolasinac is one of several players who have barely featured this season and the defender has joined Schalke on loan until the end of the campaign.</p> \n<p>“We have a large squad and we knew that,” Arteta said. “A lot of things that should have happened in the summer, we cannot accomplish for different reasons. There are some players that are going to go on loan and going to leave and that is the priority at the moment. We cannot sustain the numbers that we have in certain positions. Then we will see if we have the right opportunity in the positions where we need some more help.”</p> \n<aside class="element element-rich-link element--thumbnail"> \n <p> <span>Related: </span><a href="https://www.theguardian.com/football/2020/dec/30/from-dele-alli-to-patson-daka-eight-players-who-could-move-in-january">From Dele Alli to Patson Daka: eight players who could move in January</a> </p> \n<

In [14]:
df['bodyText'][2338]

nan

In [16]:
# use bertopic on df_women clean_text column
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(df_women['bodyText'])

In [13]:
df_women[df_women['bodyText'].apply(lambda x: not isinstance(x, str))]


,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,...,commentCloseDate,commentable,displayHint,liveBloggingNow,sensitive,starRating,scheduledPublicationDate,showTableOfContents,clean_text,womens_football
2338,football/2020/may/01/football-quiz-whose-autob...,article,football,Football,2020-05-01T09:39:17Z,Football quiz: whose autobiographies featured ...,https://www.theguardian.com/football/2020/may/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['it', 'was', 'about', 'am', 'and', 'there', '...",True
2351,football/2020/apr/30/football-quiz-identify-fi...,article,football,Football,2020-04-30T11:13:49Z,Football quiz: can you identify the fixture fr...,https://www.theguardian.com/football/2020/apr/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['frank', 'views', 'chelsea', 'v', 'west', 'ha...",True
4365,football/2021/oct/07/football-quiz-internation...,article,football,Football,2021-10-07T07:00:46Z,Football quiz: international caps,https://www.theguardian.com/football/2021/oct/...,https://content.guardianapis.com/football/2021...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['which', 'two', 'european', 'male', 'players'...",True
8740,football/2022/aug/30/football-quiz-name-the-te...,article,football,Football,2022-08-30T11:00:05Z,Football quiz: name the team from the floodlights,https://www.theguardian.com/football/2022/aug/...,https://content.guardianapis.com/football/2022...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['which', 'team', 'plays', 'under', 'these', '...",True
10598,football/2022/feb/14/valentines-day-football-quiz,article,football,Football,2022-02-14T00:01:01Z,"Valentine’s Day football quiz: roses, romance ...",https://www.theguardian.com/football/2022/feb/...,https://content.guardianapis.com/football/2022...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['the', 'photo', 'above', 'was', 'taken', 'at'...",True
12628,football/2023/aug/11/womens-world-cup-2023-how...,article,football,Football,2023-08-10T15:00:04Z,How well do you know the Matildas? Test your k...,https://www.theguardian.com/football/2023/aug/...,https://content.guardianapis.com/football/2023...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,"['who', 'has', 'scored', 'the', 'most', 'goals...",True


In [15]:
df_women = df_women.dropna(subset=['bodyText'])


In [68]:
df_men = df_men.dropna(subset=['bodyText'])

In [17]:
# display results
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1509,-1_the_to_and_of,"[the, to, and, of, in, is, for, it, that, on]","[That’s it for today. Thanks for reading, emai..."
1,0,185,0_matildas_the_australia_mins,"[matildas, the, australia, mins, her, in, to, ...",[Thank you for joining me tonight. Australia m...
2,1,153,1_she_her_england_the,"[she, her, england, the, wiegman, to, and, in,...",[After England’s World Cup semi-final defeat o...
3,2,125,2_womens_football_the_of,"[womens, football, the, of, to, and, in, that,...",[Thank you so much for joining us for this rec...
4,3,102,3_qatar_world_of_the,"[qatar, world, of, the, rights, fifa, to, and,...",[‘Too many people have died to elevate a count...
5,4,87,4_england_the_min_and,"[england, the, min, and, to, but, on, in, is, it]",[Here’s Suzanne Wrack’s report from chilly Wem...
6,5,76,5_chelsea_hayes_the_she,"[chelsea, hayes, the, she, her, min, to, and, ...","[And here it is! OK, that’s about us – thanks ..."
7,6,64,6_the_to_and_of,"[the, to, and, of, in, for, on, he, at, his]",[And that’s it for today everyone. Thanks for ...
8,7,63,7_rubiales_hermoso_spanish_federation,"[rubiales, hermoso, spanish, federation, the, ...",[Luis Rubiales has been banned from all footba...
9,8,61,8_lyon_wolfsburg_min_barcelona,"[lyon, wolfsburg, min, barcelona, the, but, in...",[Manager reaction: Jonathan Liew musings: Well...


In [18]:
topic_model.visualize_topics()

In [32]:
topic_model.visualize_hierarchy()

AttributeError: Module 'scipy' has no attribute 'array'

In [24]:
# create timestamps by taking webPublicationDate and using date
timestamps = pd.to_datetime(df_women['webPublicationDate'])

# make this into list format
timestamps = timestamps.tolist()

In [21]:
# create list of articles from bodyText
articles = df_women['bodyText'].tolist()

In [26]:
topics_over_time = topic_model.topics_over_time(articles, timestamps, datetime_format="%b%M", nr_bins=20)


In [27]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [30]:
topic_model.visualize_topics_over_time(topics_over_time, topics=[2,3,7,10])


## Testing with use of KeyBERTInspired embeddings

In [45]:
from bertopic.representation import KeyBERTInspired

# Fine-tune your topic representations
representation_model = KeyBERTInspired()
topic_model = BERTopic(representation_model=representation_model)

In [46]:
# Fit the model on your data
topics, probs = topic_model.fit_transform(df_women['bodyText'])



In [48]:
# list topics
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1433,-1_chelsea_arsenal_league_united,"[chelsea, arsenal, league, united, goal, playe...","[That’s all for today, but let’s recap some of..."
1,0,197,0_australias_matildas_sydney_australia,"[australias, matildas, sydney, australia, aust...",[Thank you for joining me tonight for another ...
2,1,159,1_englands_wiegman_england_injury,"[englands, wiegman, england, injury, williamso...",[After England’s World Cup semi-final defeat o...
3,2,113,2_football_sports_sport_womens,"[football, sports, sport, womens, women, teams...",[A lot has changed in women’s football since I...
4,3,103,3_fifa_qatar_qatari_qatars,"[fifa, qatar, qatari, qatars, fifas, arabia, f...",[‘Too many people have died to elevate a count...
...,...,...,...,...,...
56,55,12,55_eriksen_uefa_denmark_england,"[eriksen, uefa, denmark, england, danish, bruy...","[Right, it’s time to shift focus to Copenhagen..."
57,56,11,56_neville_nevilles_riise_midfielder,"[neville, nevilles, riise, midfielder, manager...",[Hege Riise became England interim manager alm...
58,57,10,57_athletes_footballers_sports_sport,"[athletes, footballers, sports, sport, womens,...",[Most people who menstruate will have a story ...
59,58,10,58_offences_alleged_bail_footballer,"[offences, alleged, bail, footballer, offence,...",[The Premier League footballer under investiga...


In [47]:
# Optionally, visualize the topics
topic_model.visualize_topics()

In [49]:
topics_over_time = topic_model.topics_over_time(articles, timestamps, datetime_format="%b%M", nr_bins=20)


In [50]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

## Testing with embedding model

In [53]:
from transformers.pipelines import pipeline

embedding_model = pipeline("feature-extraction", model="distilbert-base-cased", device=0)
topic_model = BERTopic(embedding_model=embedding_model).fit(df_women['bodyText'])

In [54]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1487,-1_the_to_and_in,"[the, to, and, in, of, it, that, for, is, on]","[Right, that’s it for the live blog today. The..."
1,0,251,0_of_the_to_fiver,"[of, the, to, fiver, and, in, on, that, for, his]","[And with that, I’ll bid you good evening. Enj..."
2,1,157,1_min_the_mins_ball,"[min, the, mins, ball, and, is, to, but, from,...",[Thank you for joining me. Here is Suzanne Wra...
3,2,120,2_min_the_but_it,"[min, the, but, it, to, and, ball, is, for, on]",[Suzanne Wrack’s report has just landed and wi...
4,3,98,3_he_was_his_the,"[he, was, his, the, and, in, it, to, had, him]",[How do you even begin to get your head around...
5,4,77,4_women_the_in_of,"[women, the, in, of, and, football, to, said, ...",[The remaining tickets for the final of the wo...
6,5,76,5_she_to_that_and,"[she, to, that, and, her, my, it, me, was, the]","[“I touched a ball, so that’s a nice feeling,”..."
7,6,63,6_police_the_of_hillsborough,"[police, the, of, hillsborough, said, that, an...",[1989 27 March The South Yorkshire police chie...
8,7,62,7_women_the_football_of,"[women, the, football, of, to, and, that, is, ...",[There was excitement and anticipation before ...
9,8,55,8_the_of_and_in,"[the, of, and, in, to, was, this, that, is, her]",[We still love you; come home. Like all good t...


In [55]:
topic_model.visualize_topics()

In [56]:
topics_over_time = topic_model.topics_over_time(articles, timestamps, datetime_format="%b%M", nr_bins=20)


In [57]:
# visualize topics over time
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

## Testing seeded topic modelling

In [60]:
seed_topic_list = [["family", "wife", "partner", "girlfriend"],
                   ["appearance", "hair", "body"],
                   ["abuse", "allegations", "treatment"]]

topic_model = BERTopic(seed_topic_list=seed_topic_list)
topics, probs = topic_model.fit_transform(df_women['bodyText'])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

# Testing sentiment classification

In [62]:

# Load the sentiment analysis pipeline with the specified model and use GPU if available
classifier = pipeline('sentiment-analysis', model="nlptown/bert-base-multilingual-uncased-sentiment", device=0)

# Function to classify sentiment with text truncation for texts longer than 512 tokens
def classify_sentiment(text):
    # Tokenization will truncate the input to 512 tokens (as required by the model)
    truncated_text = text[:512]  # Truncate the text to the first 512 tokens
    result = classifier(truncated_text)[0]  # Get the result for the truncated text
    return result['label'], result['score']  # Return the sentiment label and confidence score

# Apply the classifier to the 'bodyText' column and add the results as new columns
df_women['sentiment_label'], df_women['sentiment_score'] = zip(*df_women['bodyText'].apply(classify_sentiment))

# Display the DataFrame with sentiment classifications
df_women.head()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,...,displayHint,liveBloggingNow,sensitive,starRating,scheduledPublicationDate,showTableOfContents,clean_text,womens_football,sentiment_label,sentiment_score
5,football/2020/dec/31/aston-villa-my-crazy-chal...,article,football,Football,2020-12-31T13:45:23Z,"My crazy, challenging first year at Aston Vill...",https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,"['last', 'monday', 'aston', 'villa', 'women', ...",True,5 stars,0.447918
9,football/2020/dec/30/how-did-leeds-united-not-...,article,football,Football,2020-12-30T19:13:30Z,How did Leeds United not realise what would ha...,https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,"['when', 'the', 'former', 'england', 'internat...",True,1 star,0.408263
10,football/2020/dec/30/leeds-united-criticised-f...,article,football,Football,2020-12-30T15:30:23Z,Leeds United condemn abuse of Karen Carney aft...,https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,"['leeds', 'united', 'have', 'completely', 'con...",True,1 star,0.504399
13,football/2020/dec/29/its-tragic-swedish-womens...,article,football,Football,2020-12-29T16:24:35Z,'It's tragic': Swedish women's champions disso...,https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,"['the', 'champions', 'of', 'the', 'swedish', '...",True,1 star,0.524353
15,football/2020/dec/29/five-womens-super-league-...,article,football,Football,2020-12-29T12:07:11Z,Five Women's Super League rising stars to watc...,https://www.theguardian.com/football/2020/dec/...,https://content.guardianapis.com/football/2020...,False,pillar/sport,...,NaN,NaN,NaN,NaN,NaN,NaN,"['in', 'we', 'have', 'seen', 'stars', 'flood',...",True,3 stars,0.375978


In [63]:
df_women['sentiment_label'].value_counts()

,count
sentiment_label,
4 stars,1357
1 star,653
5 stars,520
2 stars,506
3 stars,458


In [67]:
# display headings for 1 star articles
df_women[df_women['sentiment_label'] == '1 star']['webTitle'].head(30)


,webTitle
9,How did Leeds United not realise what would happen when they targeted a woman? | Suzanne Wrack
10,Leeds United condemn abuse of Karen Carney after club tweet mocked pundit
13,'It's tragic': Swedish women's champions dissolved after title win
87,Arsenal 4-0 Everton: Women's Super League – as it happened
97,Hearts owner Ann Budge: 'No future for Scottish football if we just plug on'
118,'We've done something crazy': the football boots designed for women
131,Which player has turned out for the most teams in the same division? | The Knowledge
219,Toxicity on show at Millwall goes beyond the club and football | Sean Ingle
256,Independent W-League has golden opportunity to lead the way on equal pay | Samantha Lewis
374,Anton Ferdinand: 'I have kicked myself for years for not speaking out'


In [69]:
# Apply the classifier to the 'bodyText' column and add the results as new columns
df_men['sentiment_label'], df_men['sentiment_score'] = zip(*df_men['bodyText'].apply(classify_sentiment))

# Display the DataFrame with sentiment classifications
df_men.head()

id  \
0               football/2020/dec/31/kolasinac-on-verge-of-schalke-loan-as-arteta-targets-arsenal-clearout   
1  football/2020/dec/31/talented-charismatic-volatile-the-legend-tommy-docherty-manchester-united-scotland   
2                                                             football/2020/dec/31/tommy-docherty-obituary   
4  football/2020/dec/31/kieran-trippier-suspension-fa-completely-unjust-says-diego-simeone-atletico-madrid   
7  football/2020/dec/31/a-league-and-w-league-unbundle-from-football-australia-in-historic-moment-for-game   

      type sectionId sectionName    webPublicationDate  \
0  article  football    Football  2020-12-31T22:30:04Z   
1  article  football    Football  2020-12-31T21:06:46Z   
2  article  football    Football  2020-12-31T20:42:14Z   
4  article  football    Football  2020-12-31T19:33:39Z   
7  article  football    Football  2020-12-31T05:27:12Z   

                                                                                  webTitle  \
0                       Kolasinac joins Schalke on loan as Arteta targets Arsenal clearout   
1  Tommy Docherty: colourful, complex coach with a formidable football brain | Ewan Murray   
2                                                                  Tommy Docherty obituary   
4               Kieran Trippier's suspension by FA 'completely unjust', says Diego Simeone   
7     A-League and W-League unbundle from Football Australia in 'historic moment' for game   

                                                                                                                                webUrl  \
0               https://www.theguardian.com/football/2020/dec/31/kolasinac-on-verge-of-schalke-loan-as-arteta-targets-arsenal-clearout   
1  https://www.theguardian.com/football/2020/dec/31/talented-charismatic-volatile-the-legend-tommy-docherty-manchester-united-scotland   
2                                                             https://www.theguardian.com/football/2020/dec/31/tommy-docherty-obituary   
4  https://www.theguardian.com/football/2020/dec/31/kieran-trippier-suspension-fa-completely-unjust-says-diego-simeone-atletico-madrid   
7  https://www.theguardian.com/football/2020/dec/31/a-league-and-w-league-unbundle-from-football-australia-in-historic-moment-for-game   

                                                                                                                                     apiUrl  \
0               https://content.guardianapis.com/football/2020/dec/31/kolasinac-on-verge-of-schalke-loan-as-arteta-targets-arsenal-clearout   
1  https://content.guardianapis.com/football/2020/dec/31/talented-charismatic-volatile-the-legend-tommy-docherty-manchester-united-scotland   
2                                                             https://content.guardianapis.com/football/2020/dec/31/tommy-docherty-obituary   
4  https://content.guardianapis.com/football/2020/dec/31/kieran-trippier-suspension-fa-completely-unjust-says-diego-simeone-atletico-madrid   
7  https://content.guardianapis.com/football/2020/dec/31/a-league-and-w-league-unbundle-from-football-australia-in-historic-moment-for-game   

   isHosted      pillarId  ... displayHint liveBloggingNow sensitive  \
0     False  pillar/sport  ...         NaN             NaN       NaN   
1     False  pillar/sport  ...         NaN             NaN       NaN   
2     False  pillar/sport  ...         NaN             NaN       NaN   
4     False  pillar/sport  ...         NaN             NaN       NaN   
7     False  pillar/sport  ...         NaN             NaN       NaN   

  starRating scheduledPublicationDate showTableOfContents  \
0        NaN                      NaN                 NaN   
1        NaN                      NaN                 NaN   
2        NaN                      NaN                 NaN   
4        NaN                      NaN                 NaN   
7        NaN                      NaN                 NaN   

                                         

In [70]:
df_men['sentiment_label'].value_counts()

,count
sentiment_label,
4 stars,4260
1 star,2720
2 stars,2401
3 stars,1879
5 stars,1281


In [71]:
# calculate average sentiment for both df_women and df_men
df_women['sentiment_score'].mean()

0.38941179497383904

In [72]:
df_men['sentiment_score'].mean()

0.3786779622095888